In [48]:
import pandas as pd 
from gensim import models
from nltk.corpus import stopwords 
import numpy as np
from nltk.probability import FreqDist, MLEProbDist
import scipy

stop_words = set(stopwords.words('english')) 

maharashtra = pd.DataFrame()
for i in range(1,36):
    tempo = pd.read_csv('11_%d.csv'%(i))
    maharashtra = pd.concat([tempo,maharashtra])

In [49]:
query = list(maharashtra['QueryText'])
kccans = list(maharashtra['KCCAns'])
district = list(maharashtra['DistrictName'])
state = list(maharashtra['StateName'])

final = []
for i,w in enumerate(query):
    if query[i] != kccans[i]:
        if type(query[i]) == str:
            query[i] = query[i].replace('?','')
            query[i] = query[i].replace('(','')
            query[i] = query[i].replace(')','')

    if type(query[i]) != float:
        if type(kccans[i]) != float:
            row = [query[i].lower().strip(),kccans[i].lower().strip(),district[i].lower().strip(),state[i].lower().strip()]
            if row[0].isalpha() or len(row[0])>5:
                final.append(row)

maharashtra = pd.DataFrame(final)

main = []
for w in list(maharashtra[0]):
    main.append(w.split(' '))

new_maharashtra = []
all_words = []
for w,i in enumerate(main):
    temp = []
    for j in i:
        if j not in stop_words:
            temp.append(j)
            all_words.append(j)
    
    new_maharashtra.append([maharashtra[2][w],maharashtra[3][w]] + temp)

t = FreqDist(all_words)

model = models.Word2Vec(new_maharashtra, min_count = 1,size=50)

In [68]:
u = FreqDist(district)
v = FreqDist(state)

In [69]:
word2vec_value = []
for i in new_maharashtra:
    value = np.array([0.0 for k in range(50)])
    count = len(i)
    c = 0
    for j in i:
        c += 1
        if c in [1]:
            factor = 0.001/(0.001 + u[j]/u.N())
        if c in [2]:
            factor = 0.001/(0.001 + v[j]/v.N())
        if c in [3,4,5]:
            factor = 0.001/(0.001 + t[j]/t.N())
        value += model[j]*factor
    
    value = value/count
    word2vec_value.append(value)

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [70]:
district = 'mumbai'
state = 'maharashtra'
sent = 'asking about weather'

sent_words = [district,state] + sent.split(" ")

sent_value = np.array([0.0 for k in range(50)])

sent_new = []
count = 0
for i in sent_words:
    if i not in stop_words:
        try:
            if count in [0]:
                factor = 0.001/(0.001 + u[i]/u.N())
            if count in [1]:
                factor = 0.001/(0.001 + v[i]/v.N())
            if count in [2,3,4]:
                factor = 0.001/(0.001 + t[i]/t.N())
            sent_value += model[i]*factor
            count += 1
        except:
            continue
            count += 1

sent_value = sent_value/count

all_dist = []
for i in word2vec_value:
    dist = scipy.spatial.distance.cosine(i,sent_value)
    
    all_dist.append(dist)

k = 3
ind = np.argpartition(all_dist, k)[:k]

for i in ind:
    print new_maharashtra[i]

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


['nandurbar', 'maharashtra', 'asking', 'foabout', 'weather', 'recast']
['dhule', 'maharashtra', 'asking', 'weather', 'forecasft']
['dhule', 'maharashtra', 'asking', 'weather', 'forecasft']


In [31]:
t.B()

4166